In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
import torch.nn as nn 
from torch.nn import functional as F 
import torch 
import transformers.optimization as optim 
import torch.optim as torch_optim 
from torch.utils.data import DataLoader
from tqdm import trange, tqdm
import matplotlib.pyplot as plt 
from datasets import load_dataset 
from accelerate import Accelerator, DeepSpeedPlugin, accelerator
import pickle as pkl 
import pandas as pd
import transformers
import wandb 
import pandas as pd 

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
if device == 'cuda:0': 
  print(torch.cuda.get_device_name()) 
else:
  print(device) 

MAX_LEN = 1024

NVIDIA GeForce RTX 3090


In [2]:
tokenizer = AutoTokenizer.from_pretrained('/media/uzal/New Volume/hf_models/pythia-1b-deduped-v0')
tokenizer.pad_token = tokenizer.decode(1)
tokenizer.padding_side = 'left'
model = AutoModelForCausalLM.from_pretrained('/media/uzal/New Volume/hyperion_models/context_qna/pythia_1b_best_best', device_map=device, use_cache=True)

In [3]:
test = [f'''Context:
* The general name of energy which has to do with location relative to something else is called potential energy. In this particular case, of course, we call it gravitational potential energy. If it is a question of electrical forces against which we are working, instead of gravitational forces, if we are “lifting” charges away from other charges with a lot of levers, then the energy content is called electrical potential energy. The general principle is that the change in the energy is the force times the distance that the force is pushed, and that this is a change in energy in general:
* Fig. 14-3. The potential energy between two atoms as a function of the distance between them.
* Remember that the potential φ has a physical significance: it is the potential energy which a unit charge would have if brought to the specified point in space from some reference point.
* etc., which are acting with respect to one another in pairs due to forces all of which are conservative. In these circumstances the kinetic energy in the entire system is simply the sum of the kinetic energies of all of the particular atoms or planets or whatever, and the potential energy of the system is the sum, over the pairs of particles, of the potential energy of mutual interaction of a single pair, as though the others were not there. (This is really not true for molecular forces, and the formula is somewhat more complicated; it certainly is true for Newtonian gravitation, and it is true as an approximation for molecular forces. For molecular forces there is a potential energy, but it is sometimes a more complicated function of the positions of the atoms than simply a sum of terms from pairs.) In the special case of gravity, therefore, the potential energy is the sum, over all the pairs i and j, of Gmimj/rij, as was indicated in Eq. (). Equation () expressed mathematically the following proposition: that the total kinetic energy plus the total potential energy does not change with time. As the various planets wheel about, and turn and twist and so on, if we calculate the total kinetic energy and the total potential energy we find that the total remains constant.

Question: what is potential energy 
Answer:''',
f'''Context:
* be simple. Try to imagine what makes a drag on an airplane flying through the air—the air rushing over the wings, the swirling in the back, the changes going on around the fuselage, and many other complications, and you see that there is not going to be a simple law. On the other hand, it is a remarkable fact that the drag force on an airplane is approximately a constant times the square of the velocity, or F cv2.
* law that can be used in the design of airplanes, but this law is not in the same class as the basic laws of physics, and further study of it will only make it more and more complicated. A study of how the coefficient c depends on the shape of the front of the airplane is, to put it mildly, frustrating. There just is no simple law for determining the coefficient in terms of the shape of the airplane. In contrast, the law of gravitation is simple, and further study only indicates its greater simplicity.
* air—they get too heavy to be supported any longer in the updraft. As they come down, they draw a little air with them and start a downdraft. And surprisingly enough, it is easy to see that once the downdraft is started, it will maintain itself. The air now drives itself down!

Question: what makes an airplane fly 
Answer:''', 
f'''Context:
* Where do the currents come from? One possibility would be from the motion of the electrons in atomic orbits. Actually, that is not the case for iron, although it is for some materials. In addition to moving around in an atom, an electron also spins about on its own axis—something like the spin of the earth—and it is the current from this spin that gives the magnetic field in iron. (We say “something like the spin of the earth” because the question is so deep in quantum mechanics that the classical ideas do not really describe things too well.) In most substances, some electrons spin one way and some spin the other, so the magnetism cancels out, but in iron—for a mysterious reason which we will discuss later—many of the electrons are spinning with their axes lined up, and that is the source of the magnetism.
* In any case, we have found an induced atomic moment proportional to the magnetic field B and opposing it. This is diamagnetism of matter. It is this magnetic effect that is responsible for the small force on a piece of bismuth in a nonuniform magnetic field. (You could compute the force by working out the energy of the induced moments in the field and seeing how the energy changes as the material is moved into or out of the high-field region.)
* We find that the induced magnetization—the magnetic moment per unit volume—is proportional to the magnetic field. This is the phenomenon of paramagnetism. You will see that the effect is stronger at lower temperatures and weaker at higher temperatures. When we put a field on a substance, it develops, for small fields, a magnetic moment proportional to the field. The ratio of M to B (for small fields) is called the magnetic susceptibility.

Question: what causes magnetism 
Answer:''', 
f'''Context:
* Hunan First Normal University. Hunan First Normal University, founded in 1903, is a higher education institution located in Yuelu District, Changsha, Hunan Province, China.
* Profile of Hunan First Normal College. Hunan First Normal College ---the alma mater of Mao Zedong, was founded in. 1903, and can date back to the Nan Song Dynasty when Southern Changsha City. Academy was founded. Now it is a three-year normal college, and enjoying the.
* Hunan First Normal University covers a total area of 1346 mu, with more than 420,000 square meters of floor space. The university is divided into 10 colleges.
* Hunan First Normal University. Public University, Changsha City, Hunan province, China. Introduction. Hunan First Normal University - the alma mater of Mao Zedong, was founded in 1903, and can date back to the Nan Song Dynasty when Southern Changsha City Academy was founded.
* Publish your University Ranking. Established in 1903, Hunan First Normal University is a higher education institution located in the large city of Changsha (population range of 1,000,000-5,000,000 inhabitants), Hunan. Officially accredited/recognized by the Department of Education, Hunan Province, Hunan First Normal University is a coeducational higher education institution.
* As I couldn't find the Red Hotel in the Changsha hotel section, I thought I'd review it here. It's associated with the University. Just about everything else was poor. The rooms are shabby. They provide minimal toiletries once during a three day stay.

Question: how many colleges is Hunan First Normal University divided into 
Answer:''', 
'''Context:
* Neurosomatic Therapy (NST) is an integrative form of manual therapy that identifies and corrects structural and biomechanical patterns in the body that cause chronic pain. Chronic pain is traditionally treated in a way that does not address the underlying soft tissue causes of pain.
* Neurosomatic Therapy (NST) is an integrative form of manual therapy that identifies and corrects structural and biomechanical patterns in the body that cause chronic pain. Traditionally, chronic pain is treated in a way that addresses only one aspect of the condition.
* Neurosomatic Therapy (NST) is an integrative form of manual therapy that identifies and corrects structural and biomechanical patterns in the body that cause chronic pain. Chronic pain is traditionally treated in a way that does not address the underlying soft tissue causes of pain.
* Neurosomatic Therapy (NST) is an integrative form of manual therapy that identifies and corrects structural and biomechanical patterns in the body that cause chronic pain.
* Neurosomatic Therapy (NST) is an integrative form of manual therapy that identifies and corrects structural and biomechanical patterns in the body that cause chronic pain.
* Traditionally, chronic pain is treated in a way that addresses only one aspect of the condition. Whether it be massage therapy, physical therapy, or medicinal pain management, these methods fail to fully address the soft tissue causes of pain.
* NST: An Integrated Approach to Therapy. Based on decades of research, Neurosomatic Therapy is a non-invasive, soft-tissue-based method of manual therapy incorporating a variety of techniques, including massage therapy and physical therapy principles.
* Neuromuscular and/or Neurosomatic Therapy (NMT) is a very specialized form of manual therapy. Neuromuscular Therapy includes extensive training in the physiology of the nervous system and its effect on the muscular and skeletal systems.
* Neurosomatic Therapy is an integrated manual therapy method that identifies, analyzes, and treats the biomechanical and posture issues that cause chronic pain. Through a comprehensive examination and treatment protocol, Neurosomatic Therapy can not only eliminate pain but help prevent its recurrence.
* The mission of the Neurosomatic Therapy Foundation is to fund programs that increase the availability of Neurosomatic Therapy worldwide and advance the field by facilitating manual therapy research.

Question: what is neurosomatic therapy 
Answer:''']

In [4]:
with torch.no_grad(): 
	inputs = tokenizer(test, return_tensors='pt', max_length=MAX_LEN, padding='longest', truncation=True)
	out = model.generate(**inputs.to(device), max_length=MAX_LEN, do_sample=True, temperature=0.5, pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id)
	d = tokenizer.batch_decode(out)
	for i in d: 
		print(i, end='\n\n')

<|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|>Context:
* The general name of energy which has to do with location relative to something else is called potential energy. In this particular case, of course, we call it gravitational potential energy. If it is a question of electrical forces against which we are working, instead of gravitational forces, if we are “lifting” charges away from other charges with a lot of levers, then the energy content is called electrical potential energy. The general principle is that the change in the energy is the force times the distance that the